In [ ]:
import yaml
import os
from ultralytics import YOLO
import cv2
import torch
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
def write_yaml(data: dict, path: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, 'w') as file:
        yaml.dump(data, file, default_flow_style=False)

In [ ]:
class YOLOTrainer:
    def __init__(self, model_path: str):
        self.model = YOLO(model_path)

    def train(self, data_yaml: str, epochs: int, imgsz: int, batch: int):
        return self.model.train(
            data=data_yaml,
            epochs=epochs,
            imgsz=imgsz,
            batch=batch
        )

In [ ]:
class VideoFrameLoader(torch.utils.data.Dataset):
    def __init__(self, video_path):
        self.cap = cv2.VideoCapture(video_path)
        self.length = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.current = 0

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = self.cap.read()
        if not ret:
            raise IndexError("Frame index out of range or unreadable frame.")
        return frame

    def release(self):
        self.cap.release()


In [ ]:
class YOLOInference:
    def __init__(self, model):
        self.model = model

    @torch.no_grad()
    def detect(self, frame):
        results = self.model.predict(frame)
        bounding_boxes = []
        for result in results:
            for box in result.boxes.xyxy:
                bounding_boxes.append([int(x.item()) for x in box])
        return bounding_boxes


In [ ]:
class AngleAnalyzer:
    def __init__(self, regions=3):
        self.regions = regions  # e.g., 3 or 10

    def divide_regions(self, y, h, anchor):
        region_height = (h - anchor) // self.regions
        for i in range(self.regions):
            if y < anchor + (i + 1) * region_height:
                return i
        return self.regions - 1

    def extract_angles(self, frame, bounding_boxes, anchor, height):
        region_angles = {f'region_{i+1}': [] for i in range(self.regions)}
        for box in bounding_boxes:
            x1, y1, x2, y2 = box
            roi = frame[y1:y2, x1:x2]
            try:
                gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                blurred = cv2.GaussianBlur(gray, (5, 5), 0)
                binary = cv2.adaptiveThreshold(blurred, 255,
                                               cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                               cv2.THRESH_BINARY_INV, 11, 2)
                contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                if contours:
                    contour = max(contours, key=cv2.contourArea).reshape(-1, 2)
                    if contour.shape[0] >= 2:
                        pca = PCA(n_components=2)
                        pca.fit(contour)
                        angle = np.arctan2(pca.components_[0, 1], pca.components_[0, 0]) * 180 / np.pi
                        if angle < 0:
                            angle = 180 - abs(angle)
                        center_y = (y1 + y2) // 2
                        region_idx = self.divide_regions(center_y, height, anchor)
                        region_key = f'region_{region_idx+1}'
                        region_angles[region_key].append(angle)
            except Exception:
                continue
        return region_angles


In [ ]:
def process_video(
    video_path, out_path, yolo_model, region_count=3, frame_step=1, draw=True
):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    out = cv2.VideoWriter(out_path, fourcc, fps, (width, height))

    inference = YOLOInference(yolo_model)
    analyzer = AngleAnalyzer(region_count)

    anchor = None
    frames_angles = {}
    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_idx % frame_step == 0:
            boxes = inference.detect(frame)
            if not boxes:
                out.write(frame)
                frame_idx += 1
                continue
            if anchor is None:
                anchor = min([box[1] for box in boxes])
            region_angles = analyzer.extract_angles(frame, boxes, anchor, height)
            frames_angles[frame_idx] = region_angles
            if draw:
                for box in boxes:
                    x1, y1, x2, y2 = box
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                # Draw region lines
                region_height = (height - anchor) // region_count
                for i in range(1, region_count):
                    y_line = anchor + i * region_height
                    cv2.line(frame, (0, y_line), (width, y_line), (255, 0, 0), 2)
            out.write(frame)
        frame_idx += 1
    cap.release()
    out.release()
    return frames_angles


In [ ]:
def plot_region_kde(frames_angles, regions, frame_mod=100):
    for frame, region_dict in frames_angles.items():
        if frame % frame_mod == 0 or frame == 0:
            plt.figure(figsize=(8, 6))
            for i in range(regions):
                region_key = f'region_{i+1}'
                abs_angles = [abs(a) for a in region_dict.get(region_key, [])]
                if abs_angles:
                    sns.kdeplot(abs_angles, label=f'Region {i+1} KDE')
            plt.title(f"KDE of Angles per Region, Frame {frame}")
            plt.xlabel("Angle (degrees)")
            plt.ylabel("Density")
            plt.xlim(0, 180)
            plt.legend()
            plt.grid(True)
            plt.tight_layout()
            plt.show()


In [ ]:
# 1. Write YAML data settings
write_yaml(
    {
        'train': 'C:/Users/Predator/Desktop/gpu_env_folder/AA_PProject/dataset_red&trans/images/train',
        'val': 'C:/Users/Predator/Desktop/gpu_env_folder/AA_PProject/dataset_red&trans/images/val',
        'nc': 1,
        'names': {'0': 'non-spherical'}
    },
    'C:/Users/Predator/Desktop/gpu_env_folder/AA_PProject/dataset_red&trans/data3.yaml'
)

# 2. Train YOLOv8 model (if needed)
trainer = YOLOTrainer('yolov8s.pt')
trainer.train(
    data_yaml='C:/Users/Predator/Desktop/gpu_env_folder/AA_PProject/dataset_red&trans/data3.yaml',
    epochs=110,
    imgsz=640,
    batch=32
)

# 3. Process a video and analyze angle regions
frames_angles = process_video(
    video_path=r'C:\Users\Predator\Desktop\gpu_env_folder\AA_PProject\videos\fr24_start30sec.mp4',
    out_path=r'C:\Users\Predator\Desktop\gpu_env_folder\AA_PProject\output\fr24_start30sec_3div.mp4',
    yolo_model=trainer.model,
    region_count=3,
    frame_step=1,
    draw=True
)

# 4. KDE Plot (change "region_count" for different splits)
plot_region_kde(frames_angles, regions=3)
